# Final Project

## Initial Thoughts:
- Merge football teams with duplicate names
    - Teams that move locations
    - Teams that do no have the same team name (the ID of the favorite team may not always refer to the same team)
- Look at data to see which columns have enough data to conduct data analysis on
    - Some games have no weather data
    - Some stadiums have a lot of null information. Others are closed
- Need to decide what year to cutoff the data (relevance)

In [133]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [135]:
spreadspoke_df = pd.read_csv('NFL_sportsbetting_data/spreadspoke_scores.csv')
teams_df = pd.read_csv('NFL_sportsbetting_data/nfl_teams.csv')
stadiums_df = pd.read_csv('NFL_sportsbetting_data/nfl_stadiums.csv', encoding='ISO-8859-1')

# Data Cleaning

In [138]:
#List of Current NFL Teams
nfl_teams = [
    "Arizona Cardinals",  "Atlanta Falcons",  "Baltimore Ravens",  "Buffalo Bills",  "Carolina Panthers",  "Chicago Bears",
    "Cincinnati Bengals", "Cleveland Browns", "Dallas Cowboys", "Denver Broncos", "Detroit Lions", "Green Bay Packers",
    "Houston Texans", "Indianapolis Colts",  "Jacksonville Jaguars",  "Kansas City Chiefs",  "Las Vegas Raiders",
    "Los Angeles Chargers",  "Los Angeles Rams",  "Miami Dolphins",  "Minnesota Vikings",  "New England Patriots",
    "New Orleans Saints",  "New York Giants",  "New York Jets",  "Philadelphia Eagles",  "Pittsburgh Steelers",
    "San Francisco 49ers",  "Seattle Seahawks",  "Tampa Bay Buccaneers",  "Tennessee Titans",  "Washington Commanders"
]

#List of the current 30 full-time NFL Stadiums
stadiums = [
    "Acrisure Stadium", "Allegiant Stadium", "GEHA Field at Arrowhead Stadium", "AT&T Stadium", "Bank of America Stadium",
    "Caesars Superdome", "Empower Field at Mile High", "FedEx Field","FirstEnergy Stadium", "Ford Field", "Gillette Stadium",
    "Hard Rock Stadium", "Highmark Stadium", "Lambeau Field", "Levi's Stadium", "Lincoln Financial Field","Lumen Field",
    "M&T Bank Stadium", "Mercedes-Benz Stadium", "MetLife Stadium", "NRG Stadium", "Nissan Stadium", "Paycor Stadium",
    "Raymond James Stadium", "SoFi Stadium", "Soldier Field", "State Farm Stadium", "TIAA Bank Field", "U.S. Bank Stadium","Lucas Oil Stadium"
]

Spreadspoke Dataframe Data Cleaning

In [141]:
#need to make sure that data has non null values for the betting data
spreadspoke_df = spreadspoke_df.dropna(subset=['team_favorite_id','spread_favorite','over_under_line'])

# Create regex pattern to match any of the NFL team names
pattern = r'\b(' + '|'.join(nfl_teams) + r')\b'

# Filter the DataFrame to only keep rows that contain a current NFL team name
filtered_df = spreadspoke_df[spreadspoke_df['team_home'].str.contains(pattern, case=False, regex=True)]
filtered_spreadspoke_df = filtered_df[filtered_df['team_away'].str.contains(pattern, case=False, regex=True)]

# Filter the DataFrame to only keep rows after 2002
filtered_spreadspoke_df['schedule_date'] = pd.to_datetime(filtered_spreadspoke_df['schedule_date'])
df_date_filtered = filtered_spreadspoke_df[filtered_spreadspoke_df['schedule_date'] >= '2002-01-01'].reset_index()

#Filters the Dataframe to create new DF of only full-time stadiums
pattern = r'\b(' + '|'.join(stadiums) + r')\b'
spreadspoke_filtered_df = df_date_filtered[df_date_filtered['stadium'].str.contains(pattern, case=False, regex=True)].reset_index()

spreadspoke_filtered_df.head()

/var/folders/zp/q4n2k2wn0893spx4wqrssgqm0000gn/T/ipykernel_37819/3189460541.py:8: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_df = spreadspoke_df[spreadspoke_df['team_home'].str.contains(pattern, case=False, regex=True)]
/var/folders/zp/q4n2k2wn0893spx4wqrssgqm0000gn/T/ipykernel_37819/3189460541.py:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_spreadspoke_df = filtered_df[filtered_df['team_away'].str.contains(pattern, case=False, regex=True)]
/var/folders/zp/q4n2k2wn0893spx4wqrssgqm0000gn/T/ipykernel_37819/3189460541.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

,level_0,index,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
0,0,7846,2002-01-06,2001,17,False,Carolina Panthers,6.0,38.0,New England Patriots,NE,-6.5,36,Bank of America Stadium,False,35.0,6.0,79.0,NaN
1,9,7857,2002-01-06,2001,17,False,Tampa Bay Buccaneers,13.0,17.0,Philadelphia Eagles,TB,-3.0,33,Raymond James Stadium,False,62.0,11.0,78.0,NaN
2,10,7858,2002-01-06,2001,17,False,Tennessee Titans,21.0,23.0,Cincinnati Bengals,TEN,-5.5,37.5,Nissan Stadium,False,38.0,9.0,84.0,NaN
3,11,7860,2002-01-07,2001,17,False,Baltimore Ravens,19.0,3.0,Minnesota Vikings,BAL,-10.0,37.5,M&T Bank Stadium,False,35.0,10.0,98.0,NaN
4,13,7863,2002-01-13,2001,Wildcard,True,Green Bay Packers,25.0,15.0,San Francisco 49ers,GB,-3.5,39,Lambeau Field,False,NaN,NaN,NaN,NaN


Stadiums dataframe data cleaning

In [144]:
#Filters through stadiums dataframe to create new DF of only full-time stadiums
pattern = r'\b(' + '|'.join(stadiums) + r')\b'
stadiums_filtered_df = stadiums_df[stadiums_df['stadium_name'].str.contains(pattern, case=False, regex=True)].reset_index()

stadiums_filtered_df.head()

/var/folders/zp/q4n2k2wn0893spx4wqrssgqm0000gn/T/ipykernel_37819/3084909819.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  stadiums_filtered_df = stadiums_df[stadiums_df['stadium_name'].str.contains(pattern, case=False, regex=True)].reset_index()


,index,stadium_name,stadium_location,stadium_open,stadium_close,stadium_type,stadium_address,stadium_weather_station_zipcode,stadium_weather_type,stadium_capacity,stadium_surface,stadium_weather_station,stadium_weather_station_name,stadium_latitude,stadium_longitude,stadium_azimuthangle,stadium_elevation
0,0,Acrisure Stadium,"Pittsburgh, PA",2001.0,NaN,outdoor,"100 Art Rooney Ave, Pittsburgh, PA 15212",15212,cold,"65,500",Grass,USW00094823,"PITTSBURGH ASOS, PA US",40.484600,-80.214400,NaN,366.7
1,2,Allegiant Stadium,"Paradise, NV",2020.0,NaN,indoor,NaN,NaN,indoor,65000,Grass,NaN,NaN,36.090750,-115.183722,0.0,NaN
2,8,AT&T Stadium,"Arlington, TX",2009.0,NaN,retractable,"1 AT&T Way, Arlington, TX 76011",76011,indoor,"80,000",FieldTurf,USC00410337,"ARLINGTON SIX FLAGS, TX US",32.747778,-97.092778,68.0,163.4
3,11,Bank of America Stadium,"Charlotte, NC",1996.0,2016.0,outdoor,"800 S Mint St, Charlotte, NC 28202",28202,moderate,"73,778",Grass,USW00013881,"CHARLOTTE DOUGLAS AIRPORT, NC US",35.225833,-80.852778,322.4,221.9
4,14,Caesars Superdome,"New Orleans, LA",1975.0,NaN,indoor,"1500 Sugar Bowl Dr, New Orleans, LA 70112",70112,indoor,"76,468",FieldTurf,NaN,NaN,NaN,NaN,NaN,NaN


Teams dataframe data cleaning

In [147]:
# Create regex pattern to match any of the NFL team names
pattern = r'\b(' + '|'.join(nfl_teams) + r')\b'

# Filter the DataFrame to only keep rows that contain an NFL team name
teams_filtered_df = teams_df[teams_df['team_name'].str.contains(pattern, case=False, regex=True)].reset_index()

teams_filtered_df.head()

/var/folders/zp/q4n2k2wn0893spx4wqrssgqm0000gn/T/ipykernel_37819/433093956.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  teams_filtered_df = teams_df[teams_df['team_name'].str.contains(pattern, case=False, regex=True)].reset_index()


,index,team_name,team_name_short,team_id,team_id_pfr,team_conference,team_division,team_conference_pre2002,team_division_pre2002
0,0,Arizona Cardinals,Cardinals,ARI,CRD,NFC,NFC West,NFC,NFC West
1,1,Atlanta Falcons,Falcons,ATL,ATL,NFC,NFC South,NFC,NFC West
2,3,Baltimore Ravens,Ravens,BAL,RAV,AFC,AFC North,AFC,AFC Central
3,5,Buffalo Bills,Bills,BUF,BUF,AFC,AFC East,AFC,AFC East
4,6,Carolina Panthers,Panthers,CAR,CAR,NFC,NFC South,NFC,NFC West
